In [2]:
# @title Imports

import os
import mat73
import torch
import shutil
import subprocess
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from omegaconf import OmegaConf
from utils import init_random_seeds
from torch_geometric.data import download_url
from sklearn.preprocessing import StandardScaler
from utils import NEURON_LABELS, ROOT_DIR, EXPERIMENT_DATASETS
from preprocess._utils import (
    interpolate_data,
    reshape_calcium_data,
    smooth_data_preprocess,
)

# Initialize the random seeds
init_random_seeds(42)

CUDA device found.
	 GPU: NVIDIA A100 80GB PCIe


In [3]:
# @title Setup: Download the dataset

# download the dataset
dataset_name = "Dag2023"
assert dataset_name in EXPERIMENT_DATASETS, f"{dataset_name} is not a valid dataset."
config = OmegaConf.load("../configs/submodule/preprocess.yaml")
url = config.preprocess.opensource_url
zipfile = config.preprocess.opensource_zipfile
zip_path = os.path.join(ROOT_DIR, zipfile)
source_path = os.path.join(ROOT_DIR, zipfile.strip(".zip"))

if not os.path.exists(os.path.join(source_path, dataset_name)):
    download_url(url=url, folder=ROOT_DIR, filename=zipfile)
    # unzip the dataset folder
    bash_command = [
        "unzip",
        zip_path,
        "{}/*".format(dataset_name),
        "-d",
        source_path,
    ]
    std_out = subprocess.run(bash_command, text=True)  # Run the bash command
    print(std_out, end="\n\n")
    # delete the zip file
    os.unlink(zip_path)
else:
    print(f"{dataset_name} dataset already downloaded and unzipped.")

DATA_DIR = os.path.join(source_path, dataset_name)

Archive:  /net/vast-storage/scratch/vast/yanglab/qsimeon/worm-graph/opensource_data.zip
   creating: /net/vast-storage/scratch/vast/yanglab/qsimeon/worm-graph/opensource_data/Dag2023/
  inflating: /net/vast-storage/scratch/vast/yanglab/qsimeon/worm-graph/opensource_data/Dag2023/.DS_Store  
  inflating: /net/vast-storage/scratch/vast/yanglab/qsimeon/worm-graph/opensource_data/Dag2023/NeuroPAL_labels_dict.jld2  
   creating: /net/vast-storage/scratch/vast/yanglab/qsimeon/worm-graph/opensource_data/Dag2023/swf702_with_id/
  inflating: /net/vast-storage/scratch/vast/yanglab/qsimeon/worm-graph/opensource_data/Dag2023/NeuroPAL_labels_dict.json  
  inflating: /net/vast-storage/scratch/vast/yanglab/qsimeon/worm-graph/opensource_data/Dag2023/Dag2023Preprocessor_dk_edits.ipynb  
  inflating: /net/vast-storage/scratch/vast/yanglab/qsimeon/worm-graph/opensource_data/Dag2023/README.md  
   creating: /net/vast-storage/scratch/vast/yanglab/qsimeon/worm-graph/opensource_data/Dag2023/swf415_no_id/
  in